# ***File dùng để Upsert các Vector đã được lưu ngoài file JSON lên PINECONE***

In [1]:
#Tạo đường dẫn chung để đọc utils
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
# Tải các thư viện cần thiết
import json
from pinecone.grpc import PineconeGRPC as Pinecone
from dotenv import load_dotenv
import os
from utils.load_chunks_json import load_chunks_from_json

In [3]:
# Khởi tạo các biến cần thiết
load_dotenv("../.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HOST_DENSE = os.getenv("HOST_DENSE")
HOST_SPARSE = os.getenv("HOST_SPARSE")

In [10]:
# # Đọc Dense và Sparse của môn Triết Học
# TrietHoc_dense_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Dense_Global.json")
# TrietHoc_sparse_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Sparse_Global.json")

# Đọc Dense và Sparse của môn Lịch Sử Đảng
LSD_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Dense_Global.json")
LSD_sparse_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Sparse_Global.json")

# Đọc Child chunks cho Parent Document Retrieval
LSD_child_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Child_Dense_Optimal.json")

print(f"SỐ LƯỢNG CHUNKS ĐÃ TẢI:")
# print(f"  TrietHoc Dense: {len(TrietHoc_dense_vector)}")
# print(f"  TrietHoc Sparse: {len(TrietHoc_sparse_vector)}")
print(f"  LSD Dense (Parent): {len(LSD_dense_vector)}")
print(f"  LSD Sparse: {len(LSD_sparse_vector)}")
print(f"  LSD Child Dense: {len(LSD_child_dense_vector)}")

# Kiểm tra cấu trúc child chunk
if LSD_child_dense_vector:
    sample_child = LSD_child_dense_vector[0]
    print(f"\nCẤU TRÚC CHILD CHUNK:")
    print(f"  ID: {sample_child['id']}")
    print(f"  Parent ID: {sample_child['metadata'].get('parent_id', 'N/A')}")
    print(f"  Content length: {len(sample_child['metadata']['content'])} chars")
    print(f"  Vector dimension: {len(sample_child['values'])}")

SỐ LƯỢNG CHUNKS ĐÃ TẢI:
  LSD Dense (Parent): 347
  LSD Sparse: 347
  LSD Child Dense: 1971

CẤU TRÚC CHILD CHUNK:
  ID: LSD_chunk_00000_child_0
  Parent ID: LSD_chunk_00000
  Content length: 320 chars
  Vector dimension: 1024


In [11]:
pc = Pinecone(api_key=PINECONE_API_KEY)

# Lấy ra Index của Dense Database
dense_index = pc.Index(host=HOST_DENSE)

# Lấy ra Index của Sparse Database
sparse_index = pc.Index(host=HOST_SPARSE)

## ***Upsert môn Triết Học***

## Hybrid search

In [18]:
# dense_index.upsert(
#   vectors = TrietHoc_dense_vector,
#   namespace="triet-hoc"
# )
# sparse_index.upsert(
#   vectors = TrietHoc_sparse_vector,
#   namespace="triet-hoc"
# )

## Children upsert cho Parent document retrieval

In [19]:
# dense_index.upsert(
#   vectors = TrietHoc_dense_vector,
#   namespace="triet-hoc_children"
# )

## ***Upsert môn Lịch Sử Đảng***

## Hybrid search

In [12]:
dense_index.upsert(
  vectors=LSD_dense_vector,
  namespace="lich-su-dang"
)
sparse_index.upsert(
  vectors=LSD_sparse_vector,
  namespace="lich-su-dang"
)

upserted_count: 347

## Children upsert cho Parent document retrieval

In [ ]:
# Debug: Kiểm tra các biến môi trường
print("KIỂM TRA CÁC BIẾN MÔI TRƯỜNG:")
print(f"PINECONE_API_KEY: {'Có' if PINECONE_API_KEY else 'Không có'}")
print(f"HOST_DENSE: {HOST_DENSE}")
print(f"HOST_SPARSE: {HOST_SPARSE}")

print(f"\nKIỂM TRA ĐƯỜNG DẪN FILE .env:")
env_path = "../.env"
import os
print(f"File .env tại '{env_path}' tồn tại: {os.path.exists(env_path)}")
print(f"Đường dẫn tuyệt đối: {os.path.abspath(env_path)}")

# Thử đọc lại file .env
if os.path.exists(env_path):
    print(f"\nNỘI DUNG FILE .env:")
    with open(env_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if not line.strip().startswith('#') and '=' in line:
                key = line.split('=')[0]
                print(f"  {key}=<value>")
else:
    print(f"\nFile .env không tồn tại tại: {os.path.abspath(env_path)}")

🔍 KIỂM TRA CÁC BIẾN MÔI TRƯỜNG:
PINECONE_API_KEY: ✅ Có
HOST_DENSE: https://rag-ktcn-0abpzwm.svc.aped-4627-b74a.pinecone.io
HOST_SPARSE: https://ktcn-rag-sparse-0abpzwm.svc.aped-4627-b74a.pinecone.io

🔍 KIỂM TRA ĐƯỜNG DẪN FILE .env:
File .env tại '../.env' tồn tại: True
Đường dẫn tuyệt đối: d:\Documents\TDTU\Kiến tập\RAG_GeneralSubject\.env

📄 NỘI DUNG FILE .env:
  GEMINI_API_KEY=<value>
  PINECONE_API_KEY=<value>
  HOST_DENSE=<value>
  HOST_SPARSE=<value>


In [14]:
# DEBUG UPSERT ERROR
print("DEBUG UPSERT PROCESS")
print("="*50)

# 1. Kiểm tra connection
try:
    stats = dense_index.describe_index_stats()
    print("✓ Kết nối Pinecone thành công")
    print(f"Index stats: {stats}")
except Exception as e:
    print(f"❌ Lỗi kết nối Pinecone: {e}")
    exit()

# 2. Kiểm tra biến LSD_child_dense_vector
print(f"\nKIỂM TRA DỮ LIỆU:")
if 'LSD_child_dense_vector' in locals() or 'LSD_child_dense_vector' in globals():
    print(f"✓ LSD_child_dense_vector đã được load: {len(LSD_child_dense_vector)} chunks")
    
    # Kiểm tra cấu trúc chunk đầu tiên
    if LSD_child_dense_vector:
        sample = LSD_child_dense_vector[0]
        print(f"Sample chunk structure:")
        print(f"  Keys: {list(sample.keys())}")
        print(f"  ID: {sample.get('id', 'N/A')}")
        print(f"  Values length: {len(sample.get('values', []))}")
        print(f"  Metadata keys: {list(sample.get('metadata', {}).keys())}")
    else:
        print("❌ LSD_child_dense_vector rỗng")
else:
    print("❌ LSD_child_dense_vector chưa được load")

# 3. Test upsert một chunk đơn lẻ
if LSD_child_dense_vector:
    print(f"\nTEST UPSERT 1 CHUNK:")
    try:
        test_chunk = [LSD_child_dense_vector[0]]  # Chỉ lấy 1 chunk để test
        response = dense_index.upsert(
            vectors=test_chunk,
            namespace="lich-su-dang-children-test"  # Test namespace riêng
        )
        print(f"✓ Test upsert thành công: {response}")
    except Exception as e:
        print(f"❌ Test upsert thất bại: {e}")
        print(f"Error type: {type(e)}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")

# 4. Kiểm tra namespace hiện tại
print(f"\nNAMESPACE HIỆN TẠI:")
try:
    stats = dense_index.describe_index_stats()
    for namespace, info in stats.namespaces.items():
        print(f"  {namespace}: {info.vector_count} vectors")
except Exception as e:
    print(f"❌ Không thể lấy stats: {e}")

DEBUG UPSERT PROCESS
✓ Kết nối Pinecone thành công
Index stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'lich-su-dang': {'vector_count': 347},
                'triet-hoc': {'vector_count': 397}},
 'total_vector_count': 744}

KIỂM TRA DỮ LIỆU:
✓ LSD_child_dense_vector đã được load: 1971 chunks
Sample chunk structure:
  Keys: ['id', 'values', 'metadata']
  ID: LSD_chunk_00000_child_0
  Values length: 1024
  Metadata keys: ['content', 'subject', 'section', 'section_title', 'tokens', 'type', 'child_index', 'chapter', 'chapter_title', 'subsection', 'original_structure', 'parent_id', 'subsection_title']

TEST UPSERT 1 CHUNK:
✓ Test upsert thành công: upserted_count: 1


NAMESPACE HIỆN TẠI:
  lich-su-dang-children-test: 0 vectors
  lich-su-dang: 347 vectors
  triet-hoc: 397 vectors


In [15]:
# UPSERT VỚI BATCH SIZE NHỎ VÀ ERROR HANDLING
print("UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ")
print("="*50)

# Xóa namespace test trước
try:
    dense_index.delete(delete_all=True, namespace="lich-su-dang-children-test")
    print("✓ Đã xóa test namespace")
except:
    pass

# Xóa namespace chính để bắt đầu fresh
try:
    dense_index.delete(delete_all=True, namespace="lich-su-dang-children")
    print("✓ Đã xóa namespace 'lich-su-dang-children'")
except Exception as e:
    print(f"Warning: {e}")

print(f"Bắt đầu upsert {len(LSD_child_dense_vector)} chunks...")

# Upsert với batch size nhỏ hơn
batch_size = 50  # Giảm từ 100 xuống 50
total_upserted = 0
failed_batches = []

import time

for i in range(0, len(LSD_child_dense_vector), batch_size):
    batch = LSD_child_dense_vector[i:i+batch_size]
    batch_num = i // batch_size + 1
    total_batches = (len(LSD_child_dense_vector) + batch_size - 1) // batch_size
    
    try:
        print(f"Upsert batch {batch_num}/{total_batches} ({len(batch)} chunks)...", end=" ")
        
        # Upsert batch
        response = dense_index.upsert(
            vectors=batch,
            namespace="lich-su-dang-children"
        )
        
        total_upserted += response.upserted_count
        print(f"✓ {response.upserted_count}")
        
        # Kiểm tra nếu không upsert được hết
        if response.upserted_count != len(batch):
            print(f"  ⚠ Warning: Expected {len(batch)}, got {response.upserted_count}")
            failed_batches.append(batch_num)
        
        # Thêm delay nhỏ để tránh rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        failed_batches.append(batch_num)
        
        # Thử upsert từng chunk một trong batch bị lỗi
        print(f"  Trying individual chunks in batch {batch_num}...")
        for j, chunk in enumerate(batch):
            try:
                individual_response = dense_index.upsert(
                    vectors=[chunk],
                    namespace="lich-su-dang-children"
                )
                total_upserted += individual_response.upserted_count
                if j % 10 == 0:  # Print progress mỗi 10 chunks
                    print(f"    Chunk {j+1}/{len(batch)}")
            except Exception as individual_error:
                print(f"    ❌ Failed chunk {j+1}: {individual_error}")

print(f"\nKẾT QUẢ:")
print(f"Total upserted: {total_upserted}/{len(LSD_child_dense_vector)}")
print(f"Success rate: {total_upserted/len(LSD_child_dense_vector)*100:.1f}%")

if failed_batches:
    print(f"Failed batches: {failed_batches}")
else:
    print("✓ All batches successful!")

# Kiểm tra kết quả cuối cùng
print(f"\nKIỂM TRA NAMESPACE:")
stats = dense_index.describe_index_stats()
for namespace, info in stats.namespaces.items():
    if 'lich-su-dang-children' in namespace:
        print(f"  {namespace}: {info.vector_count} vectors")

if total_upserted == len(LSD_child_dense_vector):
    print(f"\n🎉 HOÀN THÀNH! Đã upsert thành công tất cả {total_upserted} chunks!")
else:
    missing = len(LSD_child_dense_vector) - total_upserted
    print(f"\n⚠ Thiếu {missing} chunks. Có thể cần retry.")

UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ
✓ Đã xóa test namespace
Bắt đầu upsert 1971 chunks...
Upsert batch 1/40 (50 chunks)... ✓ 50
Upsert batch 2/40 (50 chunks)... ✓ 50
Upsert batch 3/40 (50 chunks)... ✓ 50
Upsert batch 4/40 (50 chunks)... ✓ 50
Upsert batch 5/40 (50 chunks)... ✓ 50
Upsert batch 6/40 (50 chunks)... ✓ 50
Upsert batch 7/40 (50 chunks)... ✓ 50
Upsert batch 8/40 (50 chunks)... ✓ 50
Upsert batch 9/40 (50 chunks)... ✓ 50
Upsert batch 10/40 (50 chunks)... ✓ 50
Upsert batch 11/40 (50 chunks)... ✓ 50
Upsert batch 12/40 (50 chunks)... ✓ 50
Upsert batch 13/40 (50 chunks)... ✓ 50
Upsert batch 14/40 (50 chunks)... ✓ 50
Upsert batch 15/40 (50 chunks)... ✓ 50
Upsert batch 16/40 (50 chunks)... ✓ 50
Upsert batch 17/40 (50 chunks)... ✓ 50
Upsert batch 18/40 (50 chunks)... ✓ 50
Upsert batch 19/40 (50 chunks)... ✓ 50
Upsert batch 20/40 (50 chunks)... ✓ 50
Upsert batch 21/40 (50 chunks)... ✓ 50
Upsert batch 22/40 (50 chunks)... ✓ 50
Upsert batch 23/40 (50 chunks)... ✓ 50
Upsert batch 24/40 